### Concesión de acceso dinámico por referencia a grupos de seguridad

*Problema*
Usted tiene un grupo de aplicaciones que actualmente consiste en dos instancias y necesita permitir Secure Shell (SSH) entre ellas. Esto necesita ser configurado de una manera que permita el crecimiento futuro del número de instancias de forma segura y fácil.

*Solución*

**ADVERTENCIA**: Un error común es pensar que simplemente asociando el mismo grupo de seguridad a las ENIs para múltiples instancias EC2, permitirá la comunicación entre ellas (ver Figura).

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0206.png" width="500">

En esta receta, crearemos un grupo de seguridad y asociaremos cada uno a las ENIs de dos instancias EC2. Por último, crearemos una regla de entrada que autorice al grupo de seguridad a llegar a sí mismo en el puerto TCP 22.

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0207.png" width="500">

In [1]:
import boto3
import json
import time

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [2]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='174.17.0.0/16')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC"}])
vpc.wait_until_available()

In [3]:
# Crear routable
routetable = vpc.create_route_table()
routetable.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbookVPC-RT"}])

[ec2.Tag(resource_id='rtb-0c8ec5f83fe8094cc', key='Name', value='AWSCookbookVPC-RT')]

In [4]:
# Cree una subrede:
subnet = ec2.create_subnet(
    CidrBlock='174.17.0.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)
subnet.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-SN"}])

[ec2.Tag(resource_id='subnet-0b557c73f95eb3cf7', key='Name', value='AWSCookbook-SN')]

In [5]:
# Asociar la tabla de rutas con la subred:
routetable.associate_with_subnet(SubnetId = subnet.id)

ec2.RouteTableAssociation(id='rtbassoc-05f638bb70d03e0e0')

In [6]:
# Crear un grupo de seguridad
security_group = vpc.create_security_group(
    GroupName='AWSCookbook-SG-SSH',
    Description='Allow SSH access',
    VpcId=vpc.id
)

In [7]:
# Añada una regla de entrada al grupo de seguridad que permita el acceso en el puerto TCP 22 desde sí mismo:
ec2_client.authorize_security_group_ingress(
    GroupId=security_group.id,
    IpPermissions=[
        {'IpProtocol': 'tcp',
            'FromPort': 22,
            'ToPort': 22,
            'UserIdGroupPairs': [
                {
                    'Description': 'Allow SSH access',
                    'GroupId': security_group.id
                }
            ]
        } 
    ]
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-04be783df489252b2',
   'GroupId': 'sg-0033f0a59bae7980c',
   'GroupOwnerId': '329364475115',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'ReferencedGroupInfo': {'GroupId': 'sg-0033f0a59bae7980c',
    'UserId': '329364475115'},
   'Description': 'Allow SSH access'}],
 'ResponseMetadata': {'RequestId': '3b66f522-5ff5-4844-aee0-0615e5061285',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3b66f522-5ff5-4844-aee0-0615e5061285',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '903',
   'date': 'Tue, 04 Oct 2022 23:59:18 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

Este tipo de regla de grupo de seguridad se denomina regla de autorreferencia (*self-referencing rule*). Permite el acceso al puerto específico desde el tráfico que se origina en ENIs (no un rango estático de IPs) que tienen este mismo grupo de seguridad adjunto.

#### Crear dos instancias y conectarse mediante el gestor de sesiones SSM

In [8]:
iam = boto3.client('iam')
ssm = boto3.client('ssm')

In [9]:
# Create a role
role_name = 'Cookbook-SSM-Role'
policy_document = {
  "Version": "2012-10-17",
  "Statement": [
  {
    "Effect": "Allow",
    "Principal": {
      "Service": "ec2.amazonaws.com"
    },
    "Action": "sts:AssumeRole"
  }
  ]
}

ssm_role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(policy_document)
)

In [10]:
# Attach the policy to the role
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
)

{'ResponseMetadata': {'RequestId': '548975f0-d5a8-4b2c-b053-9629a77004b7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '548975f0-d5a8-4b2c-b053-9629a77004b7',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Tue, 04 Oct 2022 23:59:28 GMT'},
  'RetryAttempts': 0}}

In [11]:
# Create an instance profile
instance_profile_name = 'Cookbook-SSM-Profile'
instance_profile = iam.create_instance_profile(
    InstanceProfileName=instance_profile_name
)

In [12]:
# Add the role that you created to the instance profile:
iam.add_role_to_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName=role_name
)

{'ResponseMetadata': {'RequestId': '25565f08-a7ba-4d45-a8f6-d04fe8b68bb5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '25565f08-a7ba-4d45-a8f6-d04fe8b68bb5',
   'content-type': 'text/xml',
   'content-length': '228',
   'date': 'Tue, 04 Oct 2022 23:59:33 GMT'},
  'RetryAttempts': 0}}

In [13]:
# Enable Dns in VPC
vpc.modify_attribute(EnableDnsSupport={'Value': True})
vpc.modify_attribute(EnableDnsHostnames={'Value': True})

{'ResponseMetadata': {'RequestId': 'ccf6318c-7fc4-4bd2-a8be-b041b98579e0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ccf6318c-7fc4-4bd2-a8be-b041b98579e0',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '237',
   'date': 'Tue, 04 Oct 2022 23:59:35 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [14]:
# Create security group enable HHTPS:
security_group_https = vpc.create_security_group(
    GroupName='AWSCookbook-SG-HTTPS',
    Description='Allow HTTPS access',
    VpcId=vpc.id,
)

ec2_client.authorize_security_group_ingress(
    GroupId=security_group_https.id,
    IpPermissions=[
        {'FromPort': 443,
            'IpProtocol': 'tcp',
            'IpRanges': [
                {
                    'CidrIp': vpc.cidr_block,
                    'Description': 'Allow HTTPS access'
                }
            ],
            'ToPort': 443
        }
    ]
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0ca68ca9714c40f3b',
   'GroupId': 'sg-030d53b18a75f21fc',
   'GroupOwnerId': '329364475115',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 443,
   'ToPort': 443,
   'CidrIpv4': '174.17.0.0/16',
   'Description': 'Allow HTTPS access'}],
 'ResponseMetadata': {'RequestId': '2a13d3f3-63f0-4f4c-9be1-992b1b27a67c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2a13d3f3-63f0-4f4c-9be1-992b1b27a67c',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '783',
   'date': 'Tue, 04 Oct 2022 23:59:40 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [15]:
# Creates a VPC endpoint for SSM:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssm',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [16]:
# Creates a VPC endpoint for EC2 messages:
vpc_endpoint_ec2msg = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ec2messages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [17]:
# Creates a VPC endpoint for SSMMessages:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssmmessages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [18]:
def vpc_endpoint_status(vpc_endpoint_id):
    vpc_endpoint = ec2_client.describe_vpc_endpoints(
        VpcEndpointIds=[vpc_endpoint_id]
    )
    return vpc_endpoint['VpcEndpoints'][0]['State']

In [30]:
print('Waiting for VPC endpoints to be available...')
while vpc_endpoint_status(vpc_endpoint_ssm['VpcEndpoint']['VpcEndpointId']) != 'available':
    time.sleep(5)
    print('SSM endpoint is not available yet...')   
print('SSM endpoint is available')

Waiting for VPC endpoints to be available...
SSM endpoint is available


In [20]:
# Consulte en SSM el último ID de AMI de Amazon Linux 2 disponible en su región y guárdelo como variable de entorno:
ssm_response = ssm.get_parameter(
    Name='/aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2'
)

In [21]:
# Create a key pair
key_name = 'AWSCookbook-KeyPair-Instance'
keypair = ec2.create_key_pair(KeyName=key_name)

In [32]:
# Cear dos instancias EC2 que se puedan conectar a través de SSM
instances = ec2.create_instances(
    ImageId=ssm_response['Parameter']['Value'],
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName=key_name,
    SubnetId=subnet.id,
    IamInstanceProfile={
        'Name': instance_profile_name
    },
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'Cookbook-SSM-Instance'
                },
            ]
        },
    ]
)

In [33]:
# Esperar a que la instancia esté disponible
instances[-1].wait_until_running()

In [34]:
# Adjunte el grupo de seguridad a las instancias 1 y 2:
for instance in instances:
    instance.modify_attribute(Groups=[security_group.id])

In [35]:
instance_id = ssm.describe_instance_information()['InstanceInformationList'][0]['InstanceId']
print("En la terminal ejecute el siguiente comando:")
print("   aws ssm start-session --target " + instance_id)

En la terminal ejecute el siguiente comando:
   aws ssm start-session --target i-04da9e09e97594a17


En la terminal:

* Instale la utilidad Ncat:

In [ ]:
sudo yum -y install nc

Pruebe la conectividad SSH con la instancia 2 (utilice la IP de la instancia 2 que ha indicado anteriormente):

In [ ]:
nc -vz $INSTANCE_IP_2 22

Debería ver una salida similar a la siguiente:

`Ncat: Version 7.50 ( https://nmap.org/ncat )`<br>
`Ncat: Connected to 10.10.0.48:22.`<br>
`Ncat: 0 bytes sent, 0 bytes received in 0.01 seconds.`<br>
`sh-4.2$`

#### Discusión
La naturaleza bajo demanda de la nube (por ejemplo, el autoescalado) presenta una oportunidad de elasticidad. Los mecanismos de seguridad de la red, como las referencias de grupos de seguridad, son adecuados para ello. Tradicionalmente, los arquitectos de redes podían autorizar rangos CIDR dentro de las configuraciones de los cortafuegos. Este tipo de autorización se conoce generalmente como referencias estáticas. Esta práctica heredada no es escalable de forma dinámica, ya que se pueden añadir o eliminar instancias de las cargas de trabajo.

Un grupo de seguridad actúa como un cortafuegos virtual con estado para las ENIs. El comportamiento por defecto de los grupos de seguridad es bloquear implícitamente todas las entradas y permitir todas las salidas. Puede asociar varios grupos de seguridad con un ENI. Hay una cuota inicial de 5 grupos de seguridad por ENI y 60 reglas (de entrada o salida) por grupo de seguridad.

También puede especificar la notación CIDR para las autorizaciones. Por ejemplo, para una autorización destinada a permitir el acceso RDP desde su sucursal de Nueva York, utilizaría lo siguiente:

In [38]:
ec2_client.authorize_security_group_ingress(
    GroupId=security_group_https.id,
    IpPermissions=[
        {
            'FromPort': 3389,
            'IpProtocol': 'tcp',
            'IpRanges': [
                {
                    'CidrIp': '108.62.211.0/24',
                    'Description': 'Allow RDP from home'
                },
            ],
            'ToPort': 3389
        },
    ]
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-08074dd828b77769d',
   'GroupId': 'sg-030d53b18a75f21fc',
   'GroupOwnerId': '329364475115',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 3389,
   'ToPort': 3389,
   'CidrIpv4': '108.62.211.0/24',
   'Description': 'Allow RDP from home'}],
 'ResponseMetadata': {'RequestId': 'fd312a6c-0505-4b37-81fc-244482b84d21',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fd312a6c-0505-4b37-81fc-244482b84d21',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '788',
   'date': 'Wed, 05 Oct 2022 00:46:25 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}